In [1]:
from shapely.geometry import Point
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import geopandas as gpd

In [2]:
%%time
may_loc = pd.read_pickle("../data/may.pkl")
may_trip = pd.read_csv("../data/may_trip.csv")

Wall time: 5.1 s


In [3]:
#make a list of datetimes with datetime function 
start = datetime(2019, 5, 1, 0, 0)
end = datetime(2019, 5, 1, 1, 0)
delta = timedelta(hours=1)
timelist = []
timenamelist = []
while start <= end:
    timelist.append(start)
    timenamelist.append(start.strftime("%Y-%m-%d-%H-%M"))
    start += delta
# print(timelist)
# print(timenamelist)

In [4]:

#goal of function: using list of time frames, in this case, days, make a dataframe containing
#info about the location for each scooter, for each day


#make dictionary of dataframes using base dataframe and a user-defined segments_of_time_list
#from cell above
def make_day_df(datetime_list, timenames_list, dataframe, colname):
    resultslist = {} #turn into dictionary
    x = 0
    y = 1
    z = 0
    while x in range(0,len(datetime_list)-1):
        starttime = datetime_list[x]
        endtime = datetime_list[y]
        mask = (dataframe[colname] > starttime) & (dataframe[colname] <= endtime)
        framename = dataframe.loc[mask]
        x += 1
        y += 1
        print(x)
        resultslist[timenames_list[z]] = framename
        z += 1
    return resultslist
        
the_output = make_day_df(timelist, timenamelist, may_loc,"pubdatetime")

1


In [5]:
the_output.keys()

dict_keys(['2019-05-01-00-00'])

In [6]:

the_output['2019-05-01-00-00']
# the_output['2019-05-01-00-00']['sumdid'].nunique()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,93.0,0
1,2019-05-01 00:01:41.247,36.191252,-86.772945,PoweredXWRWC,35.0,0
2,2019-05-01 00:01:41.247,36.144752,-86.806293,PoweredMEJEH,90.0,0
3,2019-05-01 00:01:41.247,36.162056,-86.774688,Powered1A7TC,88.0,0
4,2019-05-01 00:01:41.247,36.150973,-86.783109,Powered2TYEF,98.0,0
...,...,...,...,...,...,...
26124,2019-05-01 00:59:59.163,36.121562,-86.774363,PoweredKJTKOKALXNPZX,81.0,3
26125,2019-05-01 00:59:59.163,36.122702,-86.775087,PoweredCVH7LMHIIIIAF,77.0,3
26126,2019-05-01 00:59:59.163,36.124224,-86.773634,Powered5SZPJJWT56FG3,81.0,3
26127,2019-05-01 00:59:59.163,36.127435,-86.776972,PoweredIKAA5T3PT44MA,42.0,3


In [13]:
promise_zones = gpd.read_file('../data/MDHA_Promise_Zones/Export_Output_5.shp')
print(promise_zones.crs)
promise_zones= promise_zones.to_crs('EPSG:4326')
print(promise_zones)

epsg:2274
   OBJECTID  ZONE_ID    SHAPE_STAr     SHAPE_STLe  \
0         1        1  1.053728e+08   49364.601403   
1         2        2  7.670585e+07   45226.018917   
2         3        3  2.985488e+08   75207.067487   
3         4        4  2.713638e+08   74348.596054   
4         5        5  3.928178e+08  106661.712102   
5         6        6  9.535996e+07   59256.923574   

                                            geometry  
0  POLYGON ((-86.76297 36.19364, -86.76297 36.193...  
1  POLYGON ((-86.77838 36.14889, -86.77823 36.148...  
2  POLYGON ((-86.70771 36.16723, -86.70758 36.166...  
3  POLYGON ((-86.74735 36.13036, -86.74728 36.130...  
4  POLYGON ((-86.78490 36.23358, -86.78452 36.232...  
5  POLYGON ((-86.85223 36.16417, -86.85204 36.164...  


In [9]:
the_output['2019-05-01-00-00']['geometry'] = the_output['2019-05-01-00-00'].apply(lambda x: Point(x['longitude'], 
                                                         x['latitude']), axis=1)
the_output['2019-05-01-00-00'].head(3)

C:\Users\PP\AppData\Local\Temp/ipykernel_22784/1146093682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  the_output['2019-05-01-00-00']['geometry'] = the_output['2019-05-01-00-00'].apply(lambda x: Point(x['longitude'],


,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,93.0,0,POINT (-86.799877 36.136822)
1,2019-05-01 00:01:41.247,36.191252,-86.772945,PoweredXWRWC,35.0,0,POINT (-86.77294500000001 36.191252)
2,2019-05-01 00:01:41.247,36.144752,-86.806293,PoweredMEJEH,90.0,0,POINT (-86.806293 36.144752)


In [14]:
one_hour_geodf = gpd.GeoDataFrame(the_output['2019-05-01-00-00'],
                           crs = promise_zones.crs,
                           geometry = the_output['2019-05-01-00-00']['geometry'])
one_hour_geodf.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,93.0,0,POINT (-86.79988 36.13682)
1,2019-05-01 00:01:41.247,36.191252,-86.772945,PoweredXWRWC,35.0,0,POINT (-86.77295 36.19125)
2,2019-05-01 00:01:41.247,36.144752,-86.806293,PoweredMEJEH,90.0,0,POINT (-86.80629 36.14475)
3,2019-05-01 00:01:41.247,36.162056,-86.774688,Powered1A7TC,88.0,0,POINT (-86.77469 36.16206)
4,2019-05-01 00:01:41.247,36.150973,-86.783109,Powered2TYEF,98.0,0,POINT (-86.78311 36.15097)


In [15]:
may_1_1AM_geodf = gpd.sjoin(one_hour_geodf, promise_zones, op="within")
may_1_1AM_geodf.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry,index_right,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,93.0,0,POINT (-86.79988 36.13682),1,2,2,7.670585e+07,45226.018917
15,2019-05-01 00:01:41.247,36.138401,-86.767465,PoweredAB4VX,65.0,0,POINT (-86.76747 36.13840),1,2,2,7.670585e+07,45226.018917
35,2019-05-01 00:01:41.247,36.147219,-86.792547,PoweredS3C46,72.0,0,POINT (-86.79255 36.14722),1,2,2,7.670585e+07,45226.018917
43,2019-05-01 00:01:41.247,36.138622,-86.793982,Powered125HI,85.0,0,POINT (-86.79398 36.13862),1,2,2,7.670585e+07,45226.018917
51,2019-05-01 00:01:41.247,36.136810,-86.799487,Powered9JD8Y,100.0,0,POINT (-86.79949 36.13681),1,2,2,7.670585e+07,45226.018917


In [16]:
may_1_1AM_geodf['ZONE_ID'].value_counts()

5    2583
2    2168
1    1761
4     884
3     424
6      48
Name: ZONE_ID, dtype: int64